# Data preprocessing

The code in this notebook takes the input pictures and applies the mediapipe hands solution to find the relevant coordinates and saves them as a .csv file which can be used by the AI model to train.

## Imports

In [1]:
!pip install mediapipe

import pandas as pd
import cv2
import mediapipe as mp
import pandas as pd

     |████████████████████████████████| 34.0 MB 49 kB/s 


## Download database

In [3]:
!pip install kaggle
# kaggle API
!echo '{"username":"spacewaiker","key":"062bca10262b87f5b8498c589cdc65c6"}' > ~/.kaggle/kaggle.json

In [4]:
# download dataset:
!kaggle datasets download grassknoted/asl-alphabet
# extract files (-q disables writing status message for every file)
!unzip -q asl-alphabet.zip
!ls

 98% 1.01G/1.03G [00:09<00:00, 90.7MB/s]
100% 1.03G/1.03G [00:09<00:00, 117MB/s] 
asl_alphabet_test  asl_alphabet_train  asl-alphabet.zip  drive	sample_data


Symbols:

|Command  |ASL letters|Sign           |
|---------|-----------|---------------|
|Line     |L          |"L" shape      |
|Ellipse  |A, S       |Fist           |
|Rectangle|G          |Point right    |
|Triangle |K, V       |Two fingers up |
|Move     |B          |Palm           |
|Delete   |R          |Crossed fingers|

## Define functions to use later

In [5]:
'''
Function to return the file names with the following format:
A/A1234.jpg     if letter = 'A'  and   i = 1234
'''
def get_file_names(letter, n=3000):
    files = []
    for i in range(1, n+1):
        files.append(f'{letter}/{letter}{i}.jpg')
    
    return files

'''
Function that applies the mediapipe hands solution to an image.
img is the image file name and hands is the hands instance to use
'''
def apply_hands(row):
    global hands
    img = row.name
    result = row.copy()

    image = cv2.imread(DIR + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    landmk = hands.process(image).multi_hand_landmarks

    if landmk:    # success
        landmarks = landmk[0]
        list_tuples = [(i.x, i.y, i.z) for i in landmarks.landmark]
        result[1:64] = [i for t in list_tuples for i in t]

    return result

In [6]:
# Mediapipe stuff
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get file names
file_names = pd.DataFrame()

file_names = file_names.append([['Line', file] for file in get_file_names('L')])
file_names = file_names.append(
    [['Ellipse', file] for file in get_file_names('A', n=1500) + get_file_names('S', n=1500)])
file_names = file_names.append([['Rectangle', file] for file in get_file_names('G')])
file_names = file_names.append(
    [['Triangle', file] for file in get_file_names('K', n=1500) + get_file_names('V', n=1500)])
file_names = file_names.append([['Move', file] for file in get_file_names('B')])
file_names = file_names.append([['Delete', file] for file in get_file_names('R')])

file_names.rename(columns={0: 'Command', 1: 'File'}, inplace=True)
file_names.set_index('File', inplace=True)
print(file_names)

            Command
File               
L/L1.jpg       Line
L/L2.jpg       Line
L/L3.jpg       Line
L/L4.jpg       Line
L/L5.jpg       Line
...             ...
R/R2996.jpg  Delete
R/R2997.jpg  Delete
R/R2998.jpg  Delete
R/R2999.jpg  Delete
R/R3000.jpg  Delete

[18000 rows x 1 columns]


In [7]:
DIR = 'asl_alphabet_train/asl_alphabet_train/'

cols = ['Command'] + [i for i in range(21*3)]
train_x = pd.DataFrame(columns=cols, index=file_names.index)

train_x['Command'] = file_names['Command']


# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    train_x = train_x.apply(apply_hands, axis=1)

print(train_x)

            Command         0         1  ...        60        61        62
File                                     ...                              
L/L1.jpg       Line  0.194117  0.781227  ...  0.156872  0.606071 -0.037719
L/L2.jpg       Line  0.206340  0.751357  ...  0.158566  0.591742 -0.067008
L/L3.jpg       Line  0.205757  0.734117  ...  0.159469  0.574912 -0.068139
L/L4.jpg       Line  0.222412  0.708068  ...  0.162554  0.569236 -0.102018
L/L5.jpg       Line  0.218702  0.703621  ...  0.166273  0.544865 -0.076625
...             ...       ...       ...  ...       ...       ...       ...
R/R2996.jpg  Delete  0.761999  0.578109  ...  0.747502  0.434022 -0.045308
R/R2997.jpg  Delete  0.760152  0.583979  ...  0.747119  0.447690 -0.047027
R/R2998.jpg  Delete  0.762270  0.593931  ...  0.743116  0.453064 -0.045474
R/R2999.jpg  Delete  0.762119  0.604612  ...  0.743128  0.459108 -0.045114
R/R3000.jpg  Delete  0.763956  0.609186  ...  0.742553  0.463149 -0.045747

[18000 rows x 64 columns

In [8]:
train_x.to_csv("drive/MyDrive/MAISFinalProject/train_x.csv")

## Preprocess testing data

In [13]:
DIR = 'asl_alphabet_test/asl_alphabet_test/'

files = ['L_test.jpg', 'A_test.jpg', 'S_test.jpg', 'G_test.jpg',
         'K_test.jpg', 'V_test.jpg', 'B_test.jpg', 'R_test.jpg']

test_x = pd.DataFrame(columns=cols, index=files)

test_x['Command'] = ['Line', 'Ellipse', 'Ellipse', 'Rectangle',
                     'Triangle', 'Triangle', 'Move', 'Delete']


# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    test_x = test_x.apply(apply_hands, axis=1)

print(test_x)

              Command         0         1  ...        60        61        62
L_test.jpg       Line  0.194117  0.781227  ...  0.156872  0.606071 -0.037719
A_test.jpg    Ellipse       NaN       NaN  ...       NaN       NaN       NaN
S_test.jpg    Ellipse  0.388679  0.652530  ...  0.304713  0.491797 -0.021766
G_test.jpg  Rectangle  0.599978  0.740450  ...  0.473065  0.757525 -0.110142
K_test.jpg   Triangle  0.491627  0.935984  ...  0.411350  0.800293 -0.094555
V_test.jpg   Triangle  0.242814  0.957717  ...  0.170084  0.777523 -0.150767
B_test.jpg       Move  0.619207  0.840809  ...  0.418085  0.324503  0.012467
R_test.jpg     Delete  0.278777  0.884406  ...  0.215812  0.700789 -0.141390

[8 rows x 64 columns]


In [15]:
test_x.to_csv('drive/MyDrive/MAISFinalProject/test_x.csv')